In [31]:
import math
import cmath
import drawSvg as draw

The idea is to have a mesh on a plane defined by a function that takes a point and returns other points in the neighbourhood.
The function will be used to fill whole plane with a mesh.

In [21]:
def genNeighbours(p: complex, distance: float):
    result = []
    d = complex(distance, 0)
    rot = cmath.rect(1, cmath.pi/3)
    result.append(p+d)
    for i in range(1, 3): # to generate in all directions (1,6)
        d *= rot
        result.append(p+d)
    return result

# genNeighbours(0, 1)

In [22]:
def draw_segments(d, pts, stroke='black'):
    px_size = d.width/d.renderWidth
    p = draw.Path(stroke_width=px_size, stroke=stroke, fill='none')
    start = pts[0]
    p.M(start.real, start.imag)
    for z in pts[1:]:
        p.L(z.real, z.imag)
    d.append(p)

In [23]:
def draw_points(d, pts, fill='black'):
    px_size = d.width/d.renderWidth
    for z in pts:
        c = draw.Circle(z.real, z.imag, px_size, fill=fill)
        d.append(c)

In [24]:
def draw_circles(d, pts, stroke='black'):
    px_size = d.width/d.renderWidth
    for (z,r) in pts:
        c = draw.Circle(z.real, z.imag, r, stroke_width=px_size, stroke=stroke, fill='none')
        d.append(c)

In [25]:
d = draw.Drawing(3, 3, origin='center', displayInline=False)
d.setRenderSize(w=100)

ngh = genNeighbours(0, 1)
draw_points(d, ngh+[0])

display(d)

In [26]:
class RectArea:
    def __init__(self, ld: complex, ru: complex):
        self.ld = ld
        self.ru = ru
        
    def isWithin(self, p: complex):
        return self.ld.real <= p.real <= self.ru.real and self.ld.imag <= p.imag <= self.ru.imag

# r = RectArea(0, 10+10j)
# r.isWithin(10+10j)

In [27]:
def closeEnough(p,q):
    return abs(p-q) < 0.001

def fillArea(r, gen, start):
    ps = [start]
    q = [start]
    while q:
        p = q.pop()
        pg = [p for p in gen(p, 1) if r.isWithin(p) and not any(q for q in ps if closeEnough(p,q))]
        ps += pg
        q += pg
    return ps
    
circle_mesh = fillArea(RectArea(0, 10+10j), genNeighbours, 0.5+0.5j)

In [28]:
d = draw.Drawing(10, 10, origin=(0,0), displayInline=False)
d.setRenderSize(w=600)

cs = map(lambda c: (c, 0.5), circle_mesh)

# draw_segments(d, circle_mesh, 'red')
draw_circles(d, cs)

display(d)

In [80]:
k=20
circle_points = []
tmp = 1
shift = 1j
rotate = cmath.rect(1, 2*cmath.pi/k)
for _ in range(0, k):
    circle_points.append(tmp+shift)
    tmp *= rotate

d = draw.Drawing(2, 2, origin=(-1,0), displayInline=False)
d.setRenderSize(w=200)
draw_segments(d, circle_points)

d

In [76]:
def conjugate(z):
    return complex(z.real, -z.imag)

def homography(z, p, q):
    return (z-p)/(z-q)

def homo_to_circle(z, a):
    return homography(z, a, conjugate(a))

Below we use homography that maps entire part of a complex plane where $im(z) \ge 0$ to a circle. What's interesting is that circles are mapped into circles
by every homography.

We use this homography to transform line of segments that approximates a circle. As a result we see other approximation of a circle.
Please note that while segments in the original approximation were evenly spaced that is not the case in the transformed figure.

In [97]:
mapped_circle_points = list(map(lambda z: homo_to_circle(z, 1+1j), circle_points))

d = draw.Drawing(2, 2, origin=(-1,-1), displayInline=False)
d.setRenderSize(w=200)
draw_segments(d, mapped_circle_points)
draw_circles(d, [(0,1)], stroke='red')

d

A circle is defined by $|z - z_0| = R$. Solve it for three distinct points of the circle given.

In [98]:
# https://github.com/peawormsworth/tools/blob/master/three_point_circle/three_point_circle.py
def three_point_circle(z1,z2,z3):
    a = 1j*(z1-z2)
    b = 1j*(z3-z2)
    if a.real:
        m1 = a.imag/a.real
        c = (z1-z2)/2
        p1 = z2+c
        b1 = p1.imag-m1*p1.real
    if b.real:
        m2 = b.imag/b.real
        d = (z3-z2)/2
        p2 = z2+d
        b2 = p2.imag-m2*p2.real
    if a.real and b.real:
        x = (b2-b1)/(m1-m2)
        y = (m2*b1-m1*b2)/(m2-m1)
    elif a.real:
        x,y = 0,b1
    elif b.real:
        x,y = 0,b2
    else:
        x,y = 0,0
    center = x+1j*y
    radius = abs(center-z1)
    return x,y,radius

Just to prove that it works: take `circle_mesh` defined above and transform it to a list of 3-points. Than convert it back.

In [99]:
# TODO